# Lindsay's Script

In [ ]:
import os
import pg8000          #pg8000 access SQL databases
import pandas as pd    #paandas will be needed to work in a dataframe
from pandas import ExcelWriter

In [ ]:
#code from Agata
#these are nice functions to open LIMS, make a query and then close LIMS after

def _connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    conn = pg8000.connect(user=user, host=host, database=database, password=password, port=port)
    return conn, conn.cursor()

def _select(cursor, query):
    cursor.execute(query)
    columns = [ d[0] for d in cursor.description ]
    return [ dict(zip(columns, c)) for c in cursor.fetchall() ]

def limsquery(query, user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    """A function that takes a string containing a SQL query, connects to the LIMS database and outputs the result."""
    conn, cursor = _connect(user, host, database, password, port)
    try:
        results = _select(cursor, query)
    finally:
        
        #THESE ARE IMPORTANT!!!!!!
        #Every query needs to be closed when done
        cursor.close()             
        conn.close()
    return results


#this last function will take our query results and put them in a dataframe so that they are easy to work with
def get_lims_dataframe(query):
    '''Return a dataframe with lims query'''
    result = limsquery(query)
    try:
        data_df = pd.DataFrame(data=result, columns=result[0].keys())
    except IndexError:
        print "Could not find results for your query."
        data_df = pd.DataFrame()
    return data_df

### User Report

In [ ]:
def user_df(P_number):
    """Saves an excel dataframe with all patched cells for a specified user in current directory.
     
    Parameters
    ----------
    P_number : A string. Corresponds to the user's P number in the form of P1, P2, P3, etc.
       
    Returns
    -------
    None
    """
    user = master_df[master_df["patched_cell_container"].str.contains(P_number)]
    user.sort_values("patched_cell_container", axis = 0, inplace = True)
    user = user.loc[:,['name', 'patched_cell_container','recording_date', 'rna_qc', 'go_no_go_63x', 'link_20x']]
    writer = pd.ExcelWriter('user_dataframe.xlsx')
    user.to_excel(writer,'Sheet1')
    return None

def cell_count(P_number, df):
    """Return the number of cells in a specified dataframe for a specified user.
     
    Parameters
    ----------
    P_number : A string. Corresponds to the user's P number in the form of P1, P2, P3, etc.
    df: a pandas dataframe
    
    Returns
    -------
    An integer cell number
    """
    user = df[df["patched_cell_container"].str.contains(P_number)]
    return user["patched_cell_container"].count()

def per_user(P_number):
    """Returns a summary of patcher metrics for a specified user.
     
    Parameters
    ----------
    P_number : A string. Corresponds to the user's P number in the form of P1, P2, P3, etc.
        
    Returns
    -------
    Returns None
    Prints a summary of patcher metrics, including total cells patched, cells that passed RNA seq QC, and QC pass percentage
    """
    print "Total cells patched:", cell_count(P_number,master_df)
    print "Cells that passed RNA seq QC:", cell_count(P_number,qcpass)
    print "QC pass percentage:", float((cell_count(P_number,qcpass)))/float((cell_count(P_number,master_df)))
    print "Cells imaged at 63x:", cell_count(P_number,imagego)

def gen_filter(df, col, cond):
    filtered = df[df[col] == cond]
    return filtered

In [ ]:
query = """
WITH do_63x(cell_id, go) AS (SELECT DISTINCT cell.id, array_to_string(array_agg(DISTINCT tag.name), ' _AND_ ') 
FROM specimens cell JOIN ephys_roi_results err ON err.id = cell.ephys_roi_result_id 
JOIN specimen_tags_specimens sptagsp ON sptagsp.specimen_id = cell.id 
JOIN specimen_tags tag ON tag.id = sptagsp.specimen_tag_id AND tag.id in (602120185,602122082) 
GROUP BY cell.id ORDER BY 1), 
imgs20 AS (WITH tag_list AS (SELECT DISTINCT ims.id AS ims_id, imst.name AS tag FROM image_series ims 
JOIN image_series_image_series_tags ims2imst ON ims2imst.image_series_id = ims.id 
JOIN image_series_tags imst ON imst.id = ims2imst.image_series_tag_id 
ORDER BY 1,2) 
SELECT DISTINCT ims.id AS image_series_id, ims.workflow_state AS workflow_state, ims.specimen_id, 
ARRAY_TO_STRING(ARRAY_AGG(DISTINCT tag_list.tag), '_AND_') AS tags FROM image_series ims 
LEFT JOIN tag_list ON tag_list.ims_id = ims.id 
WHERE ims.type = 'FocalPlaneImageSeries' AND is_stack = false 
GROUP BY ims.id, ims.workflow_state, ims.specimen_id)
SELECT DISTINCT s.patched_cell_container, err.recording_date, do_63x.go AS go_no_go_63x, CASE WHEN ra.id IS NOT NULL THEN (CASE WHEN ra.failed = 't' THEN 'failed' ELSE 'passed' END) ELSE '' END AS rna_qc, s.name,
'=HYPERLINK("http://lims2/focal_plane_image_series?id=' || imgs20.image_series_id || '")' as link_20x
FROM specimens s 
LEFT JOIN projects proj ON s.project_id = proj.id 
LEFT JOIN do_63x ON do_63x.cell_id = s.id 
JOIN specimens slice ON s.parent_id = slice.id
LEFT JOIN ephys_roi_results err ON s.ephys_roi_result_id = err.id 
LEFT JOIN rna_amplification_inputs rai on rai.sample_id = s.id 
LEFT JOIN imgs20 ON slice.id = imgs20.specimen_id 
LEFT JOIN rna_amplifications ra on ra.id = rai.rna_amplification_id 
WHERE proj.code <> 'mMPATCH' AND 
s.patched_cell_container IS NOT NULL 
AND err.recording_date > '2017-10-01'
"""

#AND (err.recording_date > '2017-10-01' OR err.recording_date IS NULL)
#AND err.recording_date > '2017-10-01'

master_df = get_lims_dataframe(query)
master_df

In [ ]:
qcpass ==  gen_filtergen_filt (master_df,'rna_qc', 'passed')
qcpass

In [ ]:
imagego = gen_filter(master_df,'go_no_go_63x', '63x go')

In [ ]:
user_df('PA') #Outputs an excel sheet based on 'User'

In [ ]:
per_user('PA') #Outputs print commands with results based on 'User'